# 2_cloud_data_wh_redshift_boto3
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/93/Amazon_Web_Services_Logo.svg/2560px-Amazon_Web_Services_Logo.svg.png" width="100" height="100">

In [ ]:
import json
import configparser

import pandas as pd

#!pip install boto3
import boto3
%load_ext sql

# 1. Setup
- Create a **new IAM user**.
- Under **Attach existing policies directly**, assign **`AdministratorAccess`**.
- Store the **access key** and **secret** in  **2_cloud_data_wh_redshift_boto3.cfg** file (same folder as this notebook).
- Fill in the configuration file as follows:
    ```
    [AWS]
    KEY=YOUR_AWS_KEY
    SECRET=YOUR_AWS_SECRET
    ```

## 1.1. Troubleshoot  
If your **keys are not working**, such as encountering an **InvalidAccessKeyId** error, create another IAM user with Admin access, or follow these steps to **create a new pair of access keys**.  
1. Go to the **[IAM Dashboard](https://console.aws.amazon.com/iam/home)**.  
2. View the details of the **Admin user** you created.  
3. Select **Security Credentials** → **Create access key**.  
4. A new **Access Key ID** and **Secret** will be **generated**.  
5. Update the `.cfg` file with the **new** credentials.

## 1.2. Load configuration from a file

In [ ]:
# Load configuration file
config = configparser.ConfigParser()
config.read_file(open("dwh.cfg"))

# AWS Credentials
aws_key = config.get("AWS", "KEY")
aws_secret = config.get("AWS", "SECRET")

# AWS Region
dwh_region = config.get("DWH", "DWH_REGION")

# IAM Role
dwh_iam_role_name = config.get("DWH", "DWH_IAM_ROLE_NAME")

# Redshift Cluster Configuration
dwh_cluster_type = config.get("DWH", "DWH_CLUSTER_TYPE")
dwh_node_type = config.get("DWH", "DWH_NODE_TYPE")
dwh_num_nodes = config.get("DWH", "DWH_NUM_NODES")
dwh_cluster_identifier = config.get("DWH", "DWH_CLUSTER_IDENTIFIER")

# Database Configuration
dwh_db = config.get("DWH", "DWH_DB")
dwh_db_user = config.get("DWH", "DWH_DB_USER")
dwh_db_password = config.get("DWH", "DWH_DB_PASSWORD")
dwh_port = config.get("DWH", "DWH_PORT")

# Display parameters in a DataFrame
pd.DataFrame(
    {
        "Param": [
            "DWH_REGION",
            "DWH_IAM_ROLE_NAME",
            "DWH_CLUSTER_TYPE",
            "DWH_NODE_TYPE",
            "DWH_NUM_NODES",
            "DWH_CLUSTER_IDENTIFIER",
            "DWH_DB",
            "DWH_DB_USER",
            "DWH_DB_PASSWORD",
            "DWH_PORT",
        ],
        "Value": [
            dwh_region,
            dwh_iam_role_name,
            dwh_cluster_type,
            dwh_node_type,
            dwh_num_nodes,
            dwh_cluster_identifier,
            dwh_db,
            dwh_db_user,
            dwh_db_password,
            dwh_port,
        ],
    }
)

# 2. Initialize AWS services
with credentials from the config file. Choose the same **us-west-2** region in the AWS Web Console to see these resources.

In [ ]:
iam = boto3.client(
    "iam",
    region_name=dwh_region,
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret,
)

ec2 = boto3.resource(
    "ec2",
    region_name=dwh_region,
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret,
)

s3 = boto3.resource(
    "s3",
    region_name=dwh_region,
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret,
)

redshift = boto3.client(
    "redshift",
    region_name=dwh_region,
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret,
)

## 2.1. IAM Role Setup
Create an IAM role that allows Redshift to access S3 bucket (ReadOnly)

In [ ]:
# Create the role
try:
    print("1.1 Creating a new IAM Role")
    dwh_role = iam.create_role(
        Path="/",
        RoleName=dwh_iam_role_name,
        Description="Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps({
            "Statement": [{
                "Action": "sts:AssumeRole",
                "Effect": "Allow",
                "Principal": {"Service": "redshift.amazonaws.com"}
            }],
            "Version": "2012-10-17"
        })
    )
except Exception as e:
    print(e)

# Attach Policy
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=dwh_iam_role_name,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

# Get and print the IAM role ARN
print("1.3 Get the IAM role ARN")
role_arn = iam.get_role(RoleName=dwh_iam_role_name)["Role"]["Arn"]

print(role_arn)

## 2.2. S3: check sample data, verify dataset presence.  
The code lists and prints objects in the S3 bucket "udacity" that have keys starting with "tickets".

In [ ]:
sample_db_bucket = s3.Bucket("udacity-labs")

for obj in sample_db_bucket.objects.filter(Prefix="tickets"):
    print(obj)
    
# Uncomment the following lines to list all objects in the bucket
#for obj in sample_db_bucket.objects.all():
#    print(obj)

## 2.3. Redshift: create cluster  
Creates a Redshift cluster using the IAM role. For complete arguments to **create_cluster**, see [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster).

In [ ]:
try:
    response = redshift.create_cluster(
        # Parameters for hardware
        ClusterType=dwh_cluster_type,
        NodeType=dwh_node_type,
        NumberOfNodes=int(dwh_num_nodes),

        # Parameters for identifiers & credentials
        DBName=dwh_db,
        ClusterIdentifier=dwh_cluster_identifier,
        MasterUsername=dwh_db_user,
        MasterUserPassword=dwh_db_password,

        # Parameter for role (to allow S3 access)
        IamRoles=[role_arn],

        # Make the cluster publicly accessible
        PubliclyAccessible=True  
    )
except Exception as e:
    print(e)

## 2.4. Monitor cluster status
Format and display the cluster properties in a DataFrame. Run this block multiple times until **ClusterStatus** is **Available**.

In [ ]:
def pretty_redshift_props(props):
    pd.set_option("display.max_colwidth", None)
    
    keys_to_show = [
        "ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", 
        "DBName", "Endpoint", "NumberOfNodes", "VpcId", "PubliclyAccessible"
    ]
    
    x = [(k, v) for k, v in props.items() if k in keys_to_show]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

my_cluster_props = redshift.describe_clusters(ClusterIdentifier=dwh_cluster_identifier)["Clusters"][0]
pretty_redshift_props(my_cluster_props)

## 2.5 Take note of endpoint & IAM role
Do not run this unless **ClusterStatus** is **Available**.

In [ ]:
dwh_endpoint = my_cluster_props["Endpoint"]["Address"]
dwh_role_arn = my_cluster_props["IamRoles"][0]["IamRoleArn"]

print("DWH_ENDPOINT:", dwh_endpoint)
print("DWH_ROLE_ARN:", dwh_role_arn)

# 3. Open network access
## 3.1 Allow inbound traffic to Redshift by updating the security group.

In [ ]:
try:
    vpc = ec2.Vpc(id=my_cluster_props["VpcId"])
    default_sg = list(vpc.security_groups.all())[0]
    print(default_sg)

    default_sg.authorize_ingress(
        GroupName=default_sg.group_name,
        CidrIp="0.0.0.0/0",
        IpProtocol="TCP",
        FromPort=int(dwh_port),
        ToPort=int(dwh_port),
    )
except Exception as e:
    print(e)

## 3.2. PostgreSQL: connect to the cluster

Note: Udacity's original conn_string is: "postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)"
- The string creates a successful connection in the Udacity's Workspace platform.
- It fails to connect on my localhost.
- Below are alternative conn_strings that work on my localhost, with preference for alt1.
- Should alt1 fail to connect, try these alternatives:
    - alt2:  
conn_string = f"postgresql://{dwh_db_user}:{dwh_db_password}@{dwh_endpoint}:{dwh_port}/{dwh_db}?sslmode=verify-full&sslrootcert=/etc/ssl/certs/amazon-trust-ca-bundle.crt"  
    - alt3:  
conn_string = f"redshift+psycopg2://{dwh_db_user}:{dwh_db_password}@{dwh_endpoint}:{dwh_port}/{dwh_db}?sslmode=verify-full&sslrootcert=system"  
    - alt4:  
conn_string = f"redshift+psycopg2://{dwh_db_user}:{dwh_db_password}@{dwh_endpoint}:{dwh_port}/{dwh_db}?sslmode=verify-full&sslrootcert=/etc/ssl/certs/amazon-trust-ca-bundle.crt"
    - alt5:  
from sqlalchemy import create_engine
conn_string = f"redshift+psycopg2://{dwh_db_user}:{dwh_db_password}@{dwh_endpoint}:{dwh_port}/{dwh_db}?sslmode=verify-full&sslrootcert=/etc/ssl/certs/amazon-trust-ca-bundle.crt"

engine = create_engine(conn_string)
conn = engine.connect()

print("Connected successfully!")

In [ ]:
conn_string = f"postgresql://{dwh_db_user}:{dwh_db_password}@{dwh_endpoint}:{dwh_port}/{dwh_db}?sslmode=verify-full&sslrootcert=system"
%sql $conn_string
%sql SELECT current_user;

# 4. Data ingestion into Redshift
## 4.1. Create tables

In [ ]:
%%sql 
DROP TABLE IF EXISTS "sporting_event_ticket";
CREATE TABLE "sporting_event_ticket" (
    "id" double precision DEFAULT nextval('sporting_event_ticket_seq') NOT NULL,
    "sporting_event_id" double precision NOT NULL,
    "sport_location_id" double precision NOT NULL,
    "seat_level" numeric(1,0) NOT NULL,
    "seat_section" character varying(15) NOT NULL,
    "seat_row" character varying(10) NOT NULL,
    "seat" character varying(10) NOT NULL,
    "ticketholder_id" double precision,
    "ticket_price" numeric(8,2) NOT NULL
);

## 4.2. Load Partitioned data into the cluster

In [ ]:
%%time
qry = """
    COPY sporting_event_ticket FROM 's3://udacity-labs/tickets/split/part'
    credentials 'aws_iam_role={}'
    gzip delimiter ';' 
    compupdate off 
    region 'us-west-2'
""".format(dwh_role_arn)

%sql $qry

## 4.3. Create Tables for the non-partitioned data

In [ ]:
%%sql
DROP TABLE IF EXISTS "sporting_event_ticket_full";
CREATE TABLE "sporting_event_ticket_full" (
    "id" double precision DEFAULT nextval('sporting_event_ticket_seq') NOT NULL,
    "sporting_event_id" double precision NOT NULL,
    "sport_location_id" double precision NOT NULL,
    "seat_level" numeric(1,0) NOT NULL,
    "seat_section" character varying(15) NOT NULL,
    "seat_row" character varying(10) NOT NULL,
    "seat" character varying(10) NOT NULL,
    "ticketholder_id" double precision,
    "ticket_price" numeric(8,2) NOT NULL
);

## 4.4. Load non-partitioned data into the cluster

In [ ]:
%%time
qry = """
    COPY sporting_event_ticket FROM 's3://udacity-labs/tickets/full/full.csv.gz'
    credentials 'aws_iam_role={}'
    gzip delimiter ';' 
    compupdate off 
    region 'us-west-2'
""".format(dwh_role_arn)

%sql $qry

# 5. Clean up AWS resources
Once done, delete the Redshift cluster and IAM role. DO NOT RUN THIS UNLESS YOU ARE SURE. We will be using these resources in the next exercises.

## 5.1. Delete cluster

In [ ]:
redshift.delete_cluster( ClusterIdentifier=dwh_cluster_identifier, SkipFinalClusterSnapshot=True)

Run the following block several times until the cluster is deleted = `ClusterNotFoundFault`.

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=dwh_cluster_identifier)['Clusters'][0]
pretty_redshift_props(myClusterProps)

## 5.2. Delete IAM role

In [ ]:
iam.detach_role_policy(RoleName=dwh_iam_role_name, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=dwh_iam_role_name)